# Function calling: *part one*
Source: YouTube [video](https://www.youtube.com/watch?v=kWo3iPDsVWU) by ***Thomas Janssen***.

## 1. Instantiate the OpenAI client
Load the OpenAI api key from the hidden `.env` file via `load_dotenv()` and instantiate the OpenAI `client` object.

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import json
load_dotenv()
client = OpenAI()
type(client)

openai.OpenAI

In [2]:
!pwd

/home/matthias/Desktop/MachineLearning/FunctionCalling


## 2. Describe the function
While the `prompt` is straightforward – *What do you want?* – the `function_descriptions` list warrants further comments:
- Each list entry is a nested `dict` / *JSON object* with 3 keys:
  - `"name"` has the *function name* as value,
  - `"descriptions"` has a *detailed description* as value and
  - `"parameters"` has a nested `dict` as value.
  - The nested dictionary under `"parameters"` has three keys:
    - `"type"` usually has the value `"object"`, indicating a JSON object.
    - `"properties"` is a nested `dict` where each `key` holds yes another `dict` with keys
      - `"type"` ($\Rightarrow$ return `type`: *integer*, *string* etc.) and
      - `"description"` ($\Rightarrow$ a short string that describes what shall be returned).
    - `"required"` is a list of those `"properties"` that **must** be returned.

In [3]:
prompt = "Whats the population of Germany?"
function_descriptions = [
    {
        "name": "get_population_data",
        "description": "Get the population data for a specific country.",
        "parameters": {
            "type": "object",
            "properties": {
                "population": {
                    "type": "integer",
                    "description": "The number of inhabitants for this country",
                },
                "country_name": {
                    "type":"string",
                    "description":"The name of the country"
                }
            },
            "required": ["population","country_name"],
        },
    }
]
# print function_descriptions
print(json.dumps(function_descriptions, indent=2))

[
  {
    "name": "get_population_data",
    "description": "Get the population data for a specific country.",
    "parameters": {
      "type": "object",
      "properties": {
        "population": {
          "type": "integer",
          "description": "The number of inhabitants for this country"
        },
        "country_name": {
          "type": "string",
          "description": "The name of the country"
        }
      },
      "required": [
        "population",
        "country_name"
      ]
    }
  }
]


## 3. Make the api call and extract the results

In [4]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{
        "role": "user",
        "content": prompt
    }],
    functions=function_descriptions,
    function_call="auto"
)
output = completion.choices[0].message
country_name = json.loads(output.function_call.arguments).get("country_name")
population = json.loads(output.function_call.arguments).get("population")
print(f"{country_name} has a population of {population}")

Germany has a population of 83783942


## Challenge: get name and height of the highest building in the world
Define the function call.

In [5]:
function_description_challenge = [
    {
        "name": "get_highest_building_data",
        "description": "Name, city, country and height (in meters) of the world's highest building.",
        "parameters": {
            "type": "object",
            "properties": {
                "building_name": {
                    "type":"string",
                    "description": "The name of the world's highest building."
                },
                "city_name": {
                    "type": "string",
                    "description": "The name of the city with the world's highest building.",
                },
                "country_name": {
                    "type": "string",
                    "description": "The name of the country with the world's highest building.",
                },
                "height": {
                    "type": "integer",
                    "description": "The height of the world's highest building in meters.",
                }
            },
            "required": ["building_name", "city_name", "country_name", "height"],
        },
    }
]
print(json.dumps(function_description_challenge, indent=2))
prompt_challenge = "Which is the highest building in the world and where is it located?"

[
  {
    "name": "get_highest_building_data",
    "description": "Name, city, country and height (in meters) of the world's highest building.",
    "parameters": {
      "type": "object",
      "properties": {
        "building_name": {
          "type": "string",
          "description": "The name of the world's highest building."
        },
        "city_name": {
          "type": "string",
          "description": "The name of the city with the world's highest building."
        },
        "country_name": {
          "type": "string",
          "description": "The name of the country with the world's highest building."
        },
        "height": {
          "type": "integer",
          "description": "The height of the world's highest building in meters."
        }
      },
      "required": [
        "building_name",
        "city_name",
        "country_name",
        "height"
      ]
    }
  }
]


Execute the function call and parse the results.

In [6]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{
        "role": "user",
        "content": prompt_challenge
    }],
    functions=function_description_challenge,
    function_call="auto"
)
output = completion.choices[0].message
building_name = json.loads(output.function_call.arguments).get("building_name")
city_name = json.loads(output.function_call.arguments).get("city_name")
country_name = json.loads(output.function_call.arguments).get("country_name")
height = json.loads(output.function_call.arguments).get("height")

print(f"With {height} meters, {building_name} in {city_name} ({country_name}) is the world's highest building.")

With 828 meters, Burj Khalifa in Dubai (United Arab Emirates) is the world's highest building.


This is amazing!<br>
$\checkmark$